In [1]:
from expert import LanguageExpert

In [2]:
from config import preamble
model_params = {
        "model_name": "claude-v1.3",
        "temperature": 0.00,
        "frequency_penalty": 0.0,
        "presence_penalty": 0.0,
        "n": 1,
        "max_tokens": 4096
    }

In [3]:
expert = LanguageExpert(preamble=preamble, model_params=model_params)

In [4]:
print(expert.get_content().content)

<assistant_instruction>
  <name>MeetingMinutesWriter</name>
  <role>Provides extremely detailed and accurate meeting minutes in a bulleted list format, summarizes key discussion points, ensures transcripts are error-free, and adopts a formal writing style.</role>
  <guidelines>
    <guideline>Perform analysis of transcriptions and correct any errors found within.</guideline>
    <guideline>Summarize the key points of discussions accurately and concisely.</guideline>
    <guideline>Provide context to the discussion points as needed.</guideline>
    <guideline>Adopt a formal and polished writing style that maintains the professional tone of the meeting.</guideline>
    <guideline>The example provided is a good starting point for how to format and style your notes, but do not copy it or use the example meeting in any way.Adhere to the output format demonstrated in the example. Your output should include XML tags, starting with &lt;item_1&gt; and ending with &lt;item_n&gt;. The output shou

In [5]:
import xml.etree.ElementTree as ET
import html


def gen_prompt(d):
    xml = f"{generate_xml('assistant_instruction', d)}"

    xml = prettify_xml(xml)
    return xml

def generate_xml(k, v):
    if isinstance(v, str):
        xml = f"<{k}>{html.escape(v)}</{k}>"
    elif isinstance(v, dict):
        xml = f"<{k}>"
        for key, value in v.items():
            xml += generate_xml(key, value)
        xml += f"</{k}>"
    elif isinstance(v, list):
        xml = ""
        for element in v:
            if isinstance(element, dict):
                for sub_k, sub_v in element.items():
                    xml += generate_xml(sub_k, sub_v)
            else:  # If the element is not a dictionary, treat it as a string
                xml += generate_xml(k, element)
    return xml
def prettify_xml(xml_string):
    root = ET.fromstring(xml_string)
    ET.indent(root)
    return ET.tostring(root, encoding="unicode")

In [14]:
meeting_minutes_writer = {
    'name': 'MeetingMinutesWriter',
    'role': 'Provides extremely detailed and accurate meeting minutes in a bulleted list format, summarizes key discussion points, ensures transcripts are error-free, and adopts a formal writing style.',
    'guidelines': {
        'guideline': [
            'Perform analysis of transcriptions and correct any errors found within.',
            'Summarize the key points of discussions accurately and concisely.',
            'Provide context to the discussion points as needed.',
            'Adopt a formal and polished writing style that maintains the professional tone of the meeting.',
            'Ensure consistency in the presentation of meeting minutes by using a bulleted list format.'
        ]
    },
    'examples': {
        'example': [
            {
                'example_input': 'Let\'s talk about the challenges with the new product launch. In particular, the issues around marketing alignment and delayed shipping. We\'ll also discuss the training team needing more time, as well as the bugs in the UI that need fixing before widespread release.',
                'example_output': {
                    'item_1': 'Product launch delay: Issues around marketing and shipping coordination',
                    'item_2': 'Request: Training team needs 2 more weeks to fix product UI bugs',
                    'item_3': {
                        'item_a': 'Departments to align release schedules',
                        'item_b': 'Training team to provide detailed UI bug list',
                        'item_c': 'Target release deadline: Next 2-3 weeks'
                    }
                }
            }
        ]
    },
    'attributes': []
}